In [1]:
import numpy as np
import torch
from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class MyANN(nn.Module):
    def __init__(self, inputs, out1, out2, class_num):
        super(MyANN, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(inputs, out1),
            nn.BatchNorm1d(out1),   # 添加批标准化
            nn.ReLU(inplace=True)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(out1, out2),
            nn.BatchNorm1d(out2),
            nn.ReLU(inplace=True)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(out2, class_num)
        )
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

## 接下来做什么？
- 数据标准化：torchvision.transforms 它提供了很多处理图像的方法：
    - transforms.ToTensor()将图片对象转换成pytorch中的Tensor，转换过程中自动标准化，即Tensor的范围为[0,1];
    - transforms.Normalize()需要传入两个数，第一个是均值，第二个是方差。公式：减均值，除方差。

In [3]:
batch_size = 32
lr = 1e-3
epochs = 10

In [4]:
trans_data = transforms.Compose([  
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

- transforms.Compose([])    将各种操作组在一起
- 三通道transforms.Normalize([a,b,c], [d,e,f])  应该把3个通道的均值方差填入。

## 下载并读取数据集

In [5]:
train_datasets = datasets.MNIST(root='./data', train=True, transform=trans_data, download=True)
test_datasets = datasets.MNIST(root='./data', train=False, transform=trans_data, download=True)

In [6]:
# 生成迭代器，传入数据集，batch size，是否shuffle
train_loader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_datasets, batch_size=batch_size, shuffle=False)

In [7]:
model = MyANN(28*28, 100, 50, 10)

In [8]:
if torch.cuda.is_available():
    model = model.cuda()

In [9]:
model.parameters

<bound method Module.parameters of MyANN(
  (fc1): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (fc2): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (fc3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
  )
)>

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [76]:
# 一个epoch的训练
def train():
    
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for x, y in train_loader:
        x = x.view(x.size(0), -1)   # reshape(x.size(0), 该维度压平)
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)            # 各个分类的概率
        loss = criterion(out, y)
        train_loss += loss
        _, pred = torch.max(out, 1) # 最大概率，对应的类别
        corr_num = pred.eq(y).sum()  # tensor 得取出结果
        acc = corr_num.item() / y.shape[0]
        # print(corr_num.item(), y.shape[0])
        train_acc += acc
        # print("acc", acc)
        loss.backward()
        optimizer.step()
    return train_loss / len(train_loader), train_acc / len(train_loader)
        

In [75]:
train()

32 32


(tensor(1.0139e-05, device='cuda:0', grad_fn=<DivBackward0>),
 0.0005333333333333334)

In [15]:
28*28

784

In [18]:
32*28

896

In [40]:
?nn.Linear

In [20]:
t = torch.FloatTensor([
    [[[1,2],[2,3],[3,4],[4,5]]],
    [[[1,3],[2,4],[3,5],[4,6]]],
    [[[1,1],[2,2],[3,3],[4,4]]],
])

In [21]:
t.size()

torch.Size([3, 1, 4, 2])

In [27]:
t.view(t.size(0),-1)

tensor([[1., 2., 2., 3., 3., 4., 4., 5.],
        [1., 3., 2., 4., 3., 5., 4., 6.],
        [1., 1., 2., 2., 3., 3., 4., 4.]])

In [28]:
?t.view

In [73]:
t

tensor([[[[1., 2.],
          [2., 3.],
          [3., 4.],
          [4., 5.]]],


        [[[1., 3.],
          [2., 4.],
          [3., 5.],
          [4., 6.]]],


        [[[1., 1.],
          [2., 2.],
          [3., 3.],
          [4., 4.]]]])